In [1]:
import torch
torch.__version__

d:\IAI\ENVIRONMENT\envs\spring\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.13.0+cu116'

In [2]:
torch.cuda.is_available()


True

In [11]:
import matplotlib_inline
import torch
from torch import nn
import matplotlib
T = 1000  # 总共产生1000个点
time = torch.arange(1, T + 1, dtype=torch.float32)
x = torch.sin(0.01 * time) + torch.normal(0, 0.2, (T,))
matplotlib.plot(time, [x], 'time', 'x', xlim=[1, 1000], figsize=(6, 3))

AttributeError: module 'matplotlib' has no attribute 'plot'

In [7]:

list("adbuo")

['a', 'd', 'b', 'u', 'o']

In [9]:
def test(file):
    with open(file,'r',encoding='utf-8')as f:


In [40]:
import re
from transformers import BertTokenizer, BertModel,BertConfig,AdamW
import numpy as np
with open('Dataset/test.txt', 'r',encoding='utf-8',errors="ignore") as rf:
    data = rf.readlines()
labels=['0','1']
tokenizer=BertTokenizer.from_pretrained('bert-base-chinese')
input_ids=[]
attention_mask=[]
token_type_ids=[]
label_id_list=[]
for item in data:
            item=re.sub('\s|\t|\n',"",item)
            label=item[0]
            text=item[1:]
            label_id=labels.index(label)
            token = tokenizer(text,add_special_tokens=True, max_length=512, padding='max_length', truncation=True)
            input_ids.append(np.array(token['input_ids']))
            attention_mask.append(np.array(token['attention_mask']))
            token_type_ids.append(np.array(token['token_type_ids']))
            label_id_list.append(label_id)
print (input_ids)

[array([ 101, 1963, 3362, 2769, 3187, 5464, 3198, 5381,  677,  744, 6859,
        981, 2209, 4692, 1168, 6821, 6956, 4510, 2512, 2769, 1377, 5543,
        833, 5314, 2124, 2802, 1724, 3215,  852, 3221,  100, 1995, 3221,
       5709, 1920, 3817, 1343, 4510, 2512, 7368, 4692, 1995, 1762, 4510,
       2512, 7368, 4717, 4708, 1995, 4385, 1762, 7478, 2382, 2552, 4563,
       4510, 2512, 4873, 7178,  671, 6956, 3187, 5464, 5635, 3353, 4510,
       2512,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,  

In [2]:
import re
import torch
from transformers import BertTokenizer, BertModel,BertConfig,AdamW
from torch.utils.data import TensorDataset,DataLoader,Dataset
import torch.nn as nn
from tqdm import tqdm
import wandb
from sklearn import metrics
import numpy as np

class Dataset(Dataset):
    def __init__(self,filename):
        self.labels=['0','1']
        self.labels_id=[0,1]
        self.input_ids=[]
        self.token_type_ids=[]
        self.attention_mask=[]
        self.label_id=[]
        self.load_data(filename)

    def load_data(self,filename):
        tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
        with open(filename, 'r',encoding='utf-8',errors="ignore") as f:
            data = f.readlines()
        
        for item in data:
            item=re.sub('\s|\t|\n',"",item)
            label=item[0]
            text=item[1:]
            label_id=self.labels.index(label)
            token = tokenizer(text,add_special_tokens=True, max_length=512, padding='max_length', truncation=True)
            self.input_ids.append(np.array(token['input_ids']))
            self.attention_mask.append(np.array(token['attention_mask']))
            self.token_type_ids.append(np.array(token['token_type_ids']))
            self.label_id.append(label_id)

    def __getitem__(self, index):
        return self.input_ids[index], self.token_type_ids[index], self.attention_mask[index], self.label_id[index]

    def __len__(self):
        return len(self.input_ids)
        
class Bertclassifier(nn.Module):
    def __init__(self,bert_config,num_labels):
        super().__init__()
        self.bert=BertModel(config=bert_config)
        self.classifier=nn.Linear(bert_config.hidden_size,num_labels)
        self.softmax=nn.functional.softmax
        self.__name__='BERT'
    def forward(self,input_ids,attention_mask,token_type_ids):
        bert_output=self.bert(input_ids=input_ids,attention_mask=attention_mask,
                              token_type_ids=token_type_ids)
        pooled=bert_output[1]
        logits=self.classifier(pooled)
        return self.softmax(logits,1)
    
def run():
    DEVICE=torch.device('cuda'if torch.cuda.is_available() else 'cpu')

    train_data=Dataset('Dataset/train.txt')
    valid_data=Dataset('Dataset/validation.txt')
    test_data=Dataset('Dataset/test.txt')
    batch_size=50
    epochs=10
    learning_rate=1e-3

    train_dataloader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
    valid_dataloader=DataLoader(valid_data,batch_size=batch_size,shuffle=True)
    test_dataloader=DataLoader(test_data,batch_size=batch_size,shuffle=True)
    bert_config=BertConfig.from_pretrained('bert-base-chinese')
    num_labels=2

    model=Bertclassifier(bert_config,num_labels).to(DEVICE)

    optimizer=AdamW(model.parameters(),lr=learning_rate)

    criterion=nn.CrossEntropyLoss()

    best_f1=0

    wandb.init(project=f"hw2",name=f"{model.__name__}",entity="zengxw21")
    wandb.config={"learning_rate":0.001,"epochs":100,"batch_size":50}
    for epoch in tqdm(range(1,epochs+1)):
        losses=0
        accuracy=0

        model.train()
        
        train_bar=tqdm(train_dataloader,ncols=100)
        for input_ids, token_type_ids, attention_mask, label_id in train_bar:
            model.zero_grad()
            train_bar.set_description('Epoch %i train' % epoch)

            output = model(
                input_ids=input_ids.to(DEVICE), 
                attention_mask=attention_mask.to(DEVICE), 
                token_type_ids=token_type_ids.to(DEVICE), 
            )

            loss=criterion(output,label_id.to(DEVICE))
            losses+=loss.item()

            pred_labels=torch.argmax(output,dim=1)
            acc=torch.sum(pred_labels==label_id.to(DEVICE)).item()/len(pred_labels)

            accuracy+=acc

            loss.backward()
            optimizer.step()
            train_bar.set_postfix(loss=loss.item(),acc=acc)
            
        train_loss = losses / len(train_dataloader)
        train_acc = accuracy / len(train_dataloader)

        print('\tTrain ACC:', train_acc, '\tLoss:', train_loss)
           
        model.eval()
        losses=0
        accuracy=0
        pred_labels = []
        true_labels = []
        valid_bar = tqdm(valid_dataloader, ncols=100)
        with torch.no_grad():
            for input_ids, token_type_ids, attention_mask, label_id  in valid_bar:
                valid_bar.set_description('Epoch %i valid' % epoch)

                output = model(
                    input_ids=input_ids.to(DEVICE), 
                    attention_mask=attention_mask.to(DEVICE), 
                    token_type_ids=token_type_ids.to(DEVICE), 
                )

                loss = criterion(output, label_id.to(DEVICE))
                losses += loss.item()

                pred_label = torch.argmax(output, dim=1)   # 预测出的label
                acc = torch.sum(pred_label == label_id.to(DEVICE)).item() / len(pred_label) #acc
                accuracy+=acc
                
                valid_bar.set_postfix(loss=loss.item(), acc=acc)

                pred_labels.extend(pred_label.cpu().numpy().tolist())
                true_labels.extend(label_id.numpy().tolist())
        
        valid_loss = losses / len(valid_dataloader)
        valid_acc = accuracy / len(valid_dataloader)
        print('\tVal ACC:', valid_acc,'\tLoss:', valid_loss)

        model.eval()
        losses=0
        accuracy=0
        pred_labels = []
        true_labels = []
        test_bar = tqdm(test_dataloader, ncols=100)
        with torch.no_grad():
            for input_ids, token_type_ids, attention_mask, label_id  in test_bar:
                test_bar.set_description('Epoch %i valid' % epoch)

                output = model(
                    input_ids=input_ids.to(DEVICE), 
                    attention_mask=attention_mask.to(DEVICE), 
                    token_type_ids=token_type_ids.to(DEVICE), 
                )

                loss = criterion(output, label_id.to(DEVICE))
                losses += loss.item()

                pred_label = torch.argmax(output, dim=1)   # 预测出的label
                acc = torch.sum(pred_label == label_id.to(DEVICE)).item() / len(pred_label) #acc
                accuracy+=acc
                
                test_bar.set_postfix(loss=loss.item(), acc=acc)

                pred_labels.extend(pred_label.cpu().numpy().tolist())
                true_labels.extend(label_id.numpy().tolist())
        
        test_loss = losses / len(test_dataloader)
        test_acc = accuracy / len(test_dataloader)
        print('\ttest ACC:', test_acc,'\tLoss:', test_loss)

        wandb.log(
            {
                
                #"train_f1": train_f1,
                "val_loss": valid_loss,
                "val_acc": valid_acc,
                #"val_f1": val_f1,
                "test_loss": test_loss,
                "test_acc": test_acc,
                #"test_f1": test_f1,
            }
        )
run()

d:\IAI\ENVIRONMENT\envs\spring\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB (GPU 0; 4.00 GiB total capacity; 3.35 GiB already allocated; 0 bytes free; 3.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF